In [2]:
import requests, json
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
# from bs4 import BeautifulSoup
import time

In [ ]:
from selenium import webdriver

In [1]:
a = time.time()

chrome_path = r"C:\Users\Yash Kumar\Downloads\chromedriver_win32\chromedriver.exe"
cnn_masterlist = []

try:
    for counter in range(1, 600):
        driver =  webdriver.Chrome(chrome_path)

        if counter == 1:
            driver.get("https://www.cnn.com/search/?q=donald%20trump&size=10&page=1&sort=newest&type=article")
            time.sleep(5)

        else:
            num = (counter - 1) * 10
            driver.get(f"https://www.cnn.com/search/?q=donald%20trump&size=10&page={counter}&from={num}&sort=newest&type=article")
            time.sleep(5)

        for i in range(1, 11):
            post = driver.find_element_by_xpath(f"/html/body/div[5]/div[2]/div/div[2]/div[2]/div/div[3]/div[{i}]/div[2]/h3/a")
            link = post.get_attribute("href")
            cnn_masterlist.append(link)
            
        driver.close()
        
        if counter % 10 == 0:
            print(counter, time.time()-a)

except:
    print("worked until page", str(counter))

print(time.time() - a)

In [ ]:
a = time.time()

chrome_path = r"C:\Users\Yash Kumar\Downloads\chromedriver_win32\chromedriver.exe"
fox_masterlist = []

try:
    for counter in range(1, 600):
        driver =  webdriver.Chrome(chrome_path)

        if counter == 1:
            driver.get("""https://www.foxnews.com/search-results/search?q=donald%20trump&ss=fn&type=story&start=0""")
            time.sleep(3)

        else:
            num = (counter - 1) * 10
            driver.get(f"""https://www.foxnews.com/search-results/search?q=donald%20trump&ss=fn&type=story&start={num}""")

        for i in range(3, 13):
            post = driver.find_element_by_xpath(f"""//*[@id="search-container"]/div[{i}]/div/div/h3/a""")
            link = post.get_attribute("href")

            if fox_masterlist == []:
                fox_masterlist.append(link)

            elif fox_masterlist[-1] != link:
                # Lots of repititive links in Fox news in searches, mostly one after the other
                fox_masterlist.append(link)

            else:
                pass

        driver.close()

        if counter % 10 == 0:
            print(counter, time.time() - a)

except:
    print("worked until page", str(counter))

print(time.time() - a)

In [58]:
CNN_trump = pd.DataFrame({'CNN_URL':cnn_masterlist})
CNN_trump.to_csv("CNN URLs.csv")

In [ ]:
Fox_trump = pd.DataFrame({'Fox_URL':fox_masterlist})
Fox_trump.to_csv("Fox URLs.csv")

In [90]:
def cnn_text_title(url):
    try:
        time.sleep(0.2)
        page = requests.get(url).text
        soup = BeautifulSoup(page, "html.parser")
        name_box = soup.find_all("div", attrs={"class": lambda L: L and L.startswith("zn-body__paragraph")})
        text = " ".join(chk.get_text(' ', strip=True) for chk in name_box)
        return [text, soup.find("h1").text]
    
    except:
        return ["", ""]
    
CNN_trump["text_title"] = CNN_trump["CNN_URL"].apply(lambda x: cnn_text_title(x))

In [91]:
def fox_text_title(url):
    try:
        time.sleep(0.2)
        page = requests.get(url).text
        soup = BeautifulSoup(page, "html.parser")
        name_box = soup.find_all("p", attrs={"class":''})
        text = " ".join(chk.get_text(' ', strip=True) for chk in name_box if len(list(chk.descendants)) == 1).replace("\xa0", " ").replace("&apos;", "")
        return [text, soup.find("h1").text]
    except:
        return ["", ""]
    
Fox_trump["text_title"] = Fox_trump["Fox_URL"].apply(lambda x: fox_text_title(x))

In [94]:
CNN_trump["title"] = CNN_trump["text_title"].apply(lambda x: x[1])
CNN_trump["text"] = CNN_trump["text_title"].apply(lambda x: x[0])

Fox_trump["title"] = Fox_trump["text_title"].apply(lambda x: "" if x == "" else x[1])
Fox_trump["text"] = Fox_trump["text_title"].apply(lambda x: "" if x == "" else x[0])

# CNN_trump.drop("")

In [98]:
CNN_trump.drop("text_title", axis=1).to_pickle("CNN Trump Articles.pkl")
Fox_trump.drop("text_title", axis=1).to_pickle("Fox Trump Articles.pkl")

In [ ]:
CNN_trump.drop("text_title", axis=1).to_csv("CNN Trump Articles.csv")
Fox_trump.drop("text_title", axis=1).to_csv("Fox Trump Articles.csv")

In [5]:
import string
from collections import Counter
# from wordcloud import WordCloud
import re
import spacy
import spacy
nlp = spacy.load('en')
from spacy.tokenizer import Tokenizer
tokenizer = Tokenizer(nlp.vocab)
import nltk
# nltk.download('punkt')



import numpy as np
import pandas as pd
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from tqdm import tqdm

[nltk_data] Downloading package punkt to /home/yash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
fox_trump = pd.read_pickle("Fox Trump Articles.pkl")
cnn_trump = pd.read_pickle("CNN Trump Articles.pkl")

In [38]:
def process(text):
    table = {ord(char): " " for char in string.punctuation}
    return [t for t in nltk.word_tokenize(str(text).lower().replace("`s", "").replace("'s", "").replace("`", "'").replace("'","").translate(table))]

stop_words = set(stopwords.words('english'))

fox_trump["title_processed"] = fox_trump["title"].apply(lambda x: [w for w in process(x) if w not in stop_words])
fox_trump["text_processed"] = fox_trump["text"].apply(lambda x: [w for w in process(x) if w not in stop_words])

cnn_trump["title_processed"] = cnn_trump["title"].apply(lambda x: [w for w in process(x) if w not in stop_words])
cnn_trump["text_processed"] = cnn_trump["text"].apply(lambda x: [w for w in process(x) if w not in stop_words])

In [49]:
def generate_ngrams_string(s, n):
    # Convert to lowercases
    s = s.lower()
    s = s.replace("'", "_")
    s = s.replace("`", "_")
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

In [40]:
def ngrams_series(s, n):
    ngrams = Counter()
    for i in range(len(s)):
        ngrams.update(generate_ngrams_string(s[i], n))
        
    return ngrams

In [50]:
fox_trump_words = ngrams_series(fox_trump["text"], 1)
cnn_trump_words = ngrams_series(cnn_trump["text"], 1)
fox_trump_bigrams = ngrams_series(fox_trump["text"], 2)
cnn_trump_bigrams = ngrams_series(cnn_trump["text"], 2)
fox_trump_trigrams = ngrams_series(fox_trump["text"], 3)
cnn_trump_trigrams = ngrams_series(cnn_trump["text"], 3)
fox_trump_quadgrams = ngrams_series(fox_trump["text"], 4)
cnn_trump_quadgrams = ngrams_series(cnn_trump["text"], 4)

In [51]:
cnn_trump_bigrams.most_common(10)

[('of the', 18626),
 ('in the', 15639),
 ('to the', 9453),
 ('on the', 6926),
 ('the president', 6858),
 ('trump s', 5701),
 ('for the', 5596),
 ('in a', 5435),
 ('it s', 5092),
 ('and the', 4942)]

In [47]:
word_count = Counter()

for i in range(len(fox_trump)):
    word_count.update(fox_trump.loc[i]["text"])
    word_count.update(fox_trump.loc[i]["title"])

for i in range(len(cnn_trump)):
    word_count.update(cnn_trump.loc[i]["text"])
    word_count.update(cnn_trump.loc[i]["title"])

rare_words = [word for word, count in word_count.items() if count == 1]

Exploratory Data Analysis

To explore the common patterns available in the data, we will try to visualize the dataset and glean some information from that.

1. Word Clouds
2. Most Frequent N-grams


Advanced Data Analysis

1. Classification models
2. Latent dirichlet analysis
3. SVD
4. PCA
5. Topic modeling
6. Clustering


In [ ]:
real_wc = WordCloud(background_color="black", width=600, height=400, max_words=60).generate(real_text)
plt.imshow(real_wc)
plt.axis("off")
plt.show()

SVM Classification models

In [7]:
doc = nlp(fox_trump["text"][3])
spacy.displacy.render(doc, style='ent', jupyter=True)

In [8]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)
review = str(" ".join([i.lemma_ for i in doc]))

In [9]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

In [10]:
for i in nlp(review):
    print(i,"=>",i.pos_)

president => NOUN
keep => VERB
up => PART
trade => NOUN
policy => NOUN
pressure => NOUN
as => ADP
many => ADJ
americans => NOUN
feel => VERB
tariff => NOUN
pressure => NOUN
; => PUNCT
kevin => NOUN
corke => NOUN
report => NOUN
from => ADP
the => DET
white => ADJ
house => NOUN
. => PUNCT
mid => ADJ
continent => NOUN
and => CCONJ
-PRON- => PUNCT
employee => NOUN
be => VERB
early => ADJ
victim => NOUN
in => ADP
trumps => NOUN
trade => VERB
war => NOUN
. => PUNCT
there => ADV
will => VERB
be => VERB
many => ADJ
more => ADJ
if => ADP
the => DET
president => NOUN
continue => VERB
to => PART
raise => VERB
import => NOUN
cost => NOUN
and => CCONJ
anger => NOUN
-PRON- => PUNCT
trading => NOUN
partner => NOUN
. => PUNCT
since => ADP
the => DET
start => NOUN
of => ADP
the => DET
year => NOUN
, => PUNCT
president => NOUN
trump => NOUN
have => VERB
announce => VERB
a => DET
string => NOUN
of => ADP
new => ADJ
tariff => NOUN
- => PUNCT
which => ADJ
, => PUNCT
let => VERB
-PRON- => PROPN
be => VERB
c

In [13]:
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [15]:
tqdm.pandas()
fox_trump["processed_text"] = fox_trump["text"].progress_apply(spacy_tokenizer)
cnn_trump["processed_text"] = cnn_trump["text"].progress_apply(spacy_tokenizer)

100%|██████████| 4230/4230 [00:34<00:00, 123.95it/s]


In [18]:
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(fox_trump["processed_text"])

NUM_TOPICS = 10
# Latent Dirichlet Allocation Model
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [19]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 



# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [20]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('north', 2734.0841545524695), ('korea', 1956.2297372689477), ('kim', 1574.452492562175), ('trump', 1394.135394767062), ('nuclear', 1250.9380159734224), ('korean', 1170.0422075498122), ('summit', 892.5959378396847), ('meet', 833.8375217029422), ('president', 798.7974089106004), ('south', 722.8927921889936)]
Topic 1:
[('trump', 1388.2656722490703), ('russian', 1321.931038805557), ('russia', 1132.9526536922015), ('meet', 913.0174812774598), ('putin', 883.2511216555944), ('election', 507.63330366143595), ('gigot', 491.40152438476144), ('president', 487.5237254049152), ('vladimir', 296.2610711822475), ('campaign', 295.72705469629744)]
Topic 2:
[('think', 11437.776431849361), ('know', 8386.371177757637), ('people', 6550.641129351487), ('trump', 6086.859456254215), ('president', 6047.942455381278), ('right', 5021.628050700651), ('like', 4905.269891403698), ('want', 4510.560956424072), ('video', 4363.042656098933), ('good', 4316.578551513869)]
Topic 3:
[('trump', 3415.953

In [21]:
# Keywords for topics clustered by Latent Semantic Indexing
print("NMF Model:")
selected_topics(nmf, vectorizer)

NMF Model:
Topic 0:
[('think', 17.157355683454067), ('know', 15.589484934988743), ('people', 11.275038395870457), ('right', 10.300077878115623), ('good', 8.341381315275493), ('want', 7.934033044964453), ('like', 7.366178233216798), ('come', 7.07683111565424), ('maccallum', 6.933557465297223), ('look', 6.5683516196658305)]
Topic 1:
[('williams', 21.593073383619437), ('think', 12.377978887387636), ('perino', 7.410137687320765), ('know', 7.106717496269899), ('right', 4.771262034880656), ('people', 4.2630556786759435), ('juan', 3.7117996209901154), ('kilmeade', 3.5530392428480595), ('want', 3.06905511965947), ('look', 2.9819955213210427)]
Topic 2:
[('guilfoyle', 24.391240183176038), ('bolling', 18.440843493113526), ('like', 7.121644616051899), ('perino', 6.505044480302982), ('right', 6.06801325801524), ('know', 5.184369561634453), ('think', 4.198641798723942), ('crosstalk', 3.6507937004291633), ('good', 3.5361770529136773), ('video', 3.334975843628088)]
Topic 3:
[('kurtz', 15.0947799490816

In [24]:
# Transforming an individual sentence
text = spacy_tokenizer("""'"In respectful and friendly terms, we spoke about the migration issue and the possibility of implementing a joint program of development and job creation in Central America and our country," Lopez Obrador wrote in his Twitter account.""")
x = lda.transform(vectorizer.transform([text]))[0]
print(x)

[0.00454727 0.00454614 0.00454632 0.00454646 0.36210268 0.00454659
 0.00454593 0.00454626 0.00454572 0.60152663]


In [23]:
fox_trump["text"][156]

'"In respectful and friendly terms, we spoke about the migration issue and the possibility of implementing a joint program of development and job creation in Central America and our country," Lopez Obrador wrote in his Twitter account. The Mexican president has called on the United States to join in a "Marshall Plan" effort to commit about $20 billion in public and private investment in Central America to create jobs, so people there wont have to emigrate. Despite their differences in background and policy, the relationship between the two leaders has been quite cordial. Lopez Obrador, who took office Dec. 1, has said he hopes to make migration a choice, not a necessity, for poor people of the region. Earlier Wednesday, Mexicos top security official said the government will close off illegal entries at its southern border with Guatemala, but didnt say exactly how the country plans to accomplish that daunting task. Interior Secretary Olga Sanchez Cordero said the new administration will

In [25]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      651.964355 -307.556580       1        1  32.296353
9       98.489487  673.452942       2        1  21.737528
6     -297.788452 -225.589828       3        1  15.522237
3      -25.534697 -710.697693       4        1  10.316970
4     -475.314575  559.653381       5        1   9.293877
0     -664.847839 -540.308899       6        1   3.953999
7      461.352325  253.021622       7        1   2.340592
5     -721.748657   74.879814       8        1   2.309263
1      177.847565 -221.578064       9        1   1.951346
8      -98.350998  203.515579      10        1   0.277835, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
10546  Default  27110.000000         trump  27110.000000  30.0000  30.0000
4573   Default   3783.000000       hannity   3783.000000  29.0000  29.0000
5723   Default  15847.000000          know  15847.000000  28.0000  28.0000
4514   Default   7036.000000       gutfeld   7036.000000  27.0000  27.0000
8734   Default  10999.000000         right  10999.000000  26.0000  26.0000
7914   Default  16979.000000     president  16979.000000  25.0000  25.0000
11276  Default   5482.000000      williams   5482.000000  24.0000  24.0000
7542   Default   4804.000000        perino   4804.000000  23.0000  23.0000
11423  Default   5846.000000           yes   5846.000000  22.0000  22.0000
6983   Default   2976.000000         north   2976.000000  21.0000  21.0000
4499   Default   4424.000000     guilfoyle   4424.000000  20.0000  20.0000
7521   Default  12872.000000        people  12872.000000  19.0000  19.0000
11167  Default   4255.000000       watters   4255.000000  18.0000  18.0000
10285  Default  18020.000000         think  18020.000000  17.0000  17.0000
11122  Default   8805.000000          want   8805.000000  16.0000  16.0000
6416   Default   3050.000000          meet   3050.000000  15.0000  15.0000
8890   Default   1898.000000       russian   1898.000000  14.0000  14.0000
8889   Default   2058.000000        russia   2058.000000  13.0000  13.0000
3283   Default   2858.000000      election   2858.000000  12.0000  12.0000
5735   Default   1933.000000         korea   1933.000000  11.0000  11.0000
368    Default   4867.000000      american   4867.000000  10.0000  10.0000
1404   Default   4327.000000      campaign   4327.000000   9.0000   9.0000
2282   Default   4650.000000       country   4650.000000   8.0000   8.0000
5980   Default  11782.000000          like  11782.000000   7.0000   7.0000
2526   Default   3120.000000          deal   3120.000000   6.0000   6.0000
5687   Default   1519.000000           kim   1519.000000   5.0000   5.0000
2383   Default   2547.000000     crosstalk   2547.000000   4.0000   4.0000
1094   Default   2369.000000       bolling   2369.000000   3.0000   3.0000
10759  Default   1681.000000  unidentified   1681.000000   2.0000   2.0000
1807   Default   5379.000000          clip   5379.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
1661   Topic10      6.606959      cherokee      9.385786   5.5348  -6.4609
625    Topic10      7.514774  assimilation     10.882642   5.5156  -6.3321
3349   Topic10     13.471237          emmy     19.821918   5.4997  -5.7484
6951   Topic10      5.149593          no--      7.610218   5.4953  -6.7101
4      Topic10      5.673886         --its      8.401074   5.4934  -6.6131
1374   Topic10     80.716630          cain    121.621479   5.4759  -3.9580
10570  Topic10    117.873540        tucker    199.968639   5.3573  -3.5794
887    Topic10     10.450509          beep     16.551473   5.4261  -6.0023
4353   Topic10     77.124166        google    212.193948   4.8738  -4.0036
6304   Topic10     16.428623   masculinity     31.584193   5.2323  -5.5500
8861   Topic10     22.480141        

In [26]:
svd_2d = TruncatedSVD(n_components=2)
data_2d = svd_2d.fit_transform(data_vectorized)

trace = go.Scattergl(
    x = data_2d[:,0],
    y = data_2d[:,1],
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    ),
    text = vectorizer.get_feature_names(),
    hovertext = vectorizer.get_feature_names(),
    hoverinfo = 'text' 
)
data = [trace]
iplot(data, filename='scatter-mode')

In [27]:
trace = go.Scattergl(
    x = data_2d[:,0],
    y = data_2d[:,1],
    mode = 'text',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    ),
    text = vectorizer.get_feature_names()
)
data = [trace]
iplot(data, filename='text-scatter-mode')

In [29]:
def spacy_bigram_tokenizer(phrase):
    doc = parser(phrase) # create spacy object
    token_not_noun = []
    notnoun_noun_list = []
    noun = ""

    for item in doc:
        if item.pos_ != "NOUN": # separate nouns and not nouns
            token_not_noun.append(item.text)
        if item.pos_ == "NOUN":
            noun = item.text
        
        for notnoun in token_not_noun:
            notnoun_noun_list.append(notnoun + " " + noun)

    return " ".join([i for i in notnoun_noun_list])

bivectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, ngram_range=(1,2))
bigram_vectorized = bivectorizer.fit_transform(fox_trump["processed_text"])


In [31]:
bi_lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_bi_lda = bi_lda.fit_transform(bigram_vectorized)

print("Bi-LDA Model:")
selected_topics(bi_lda, bivectorizer)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10
Bi-LDA Model:
Topic 0:
[('trump', 17655.095063039837), ('president', 12891.157357948097), ('think', 9762.002150878632), ('people', 8126.3360749406675), ('know', 8063.361904324637), ('want', 5222.077965778481), ('right', 5024.769114253541), ('like', 4831.526803673809), ('come', 4741.566978904765), ('good', 4602.585989475803)]
Topic 1:
[('js', 34.27405274320204), ('id', 12.049988064602411), ('var', 8.090769825223472), ('function', 6.6099765039369185), ('js src', 6.1031855255954905), ('id return', 6.10318487726884), ('document script', 6.103184314033452), ('src', 6.103184288902996), ('var js', 6.103184062006866), ('js id', 6.103183963705176)]
Topic 2:
[('north', 2689.944196108827), ('trump', 2128.142060

In [34]:
bi_dash = pyLDAvis.sklearn.prepare(bi_lda, bigram_vectorized, bivectorizer, mds='tsne')
bi_dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0     -191.683273 -120.716904       1        1  54.674439
9      -36.932652 -178.136536       2        1  24.875753
5      -86.486488  -52.681988       3        1  10.791241
2       66.161415  139.920822       4        1   5.769043
4     -185.934448   41.949116       5        1   3.009939
3      109.869003 -143.782761       6        1   0.516646
7      150.207199   20.353651       7        1   0.218336
6      -35.554379   60.074791       8        1   0.082840
8     -100.388466  168.567276       9        1   0.039899
1       36.637760  -40.177029      10        1   0.021864, topic_info=      Category          Freq                  Term         Total  loglift  \
term                                                                        
42770  Default  27016.000000                 trump  27016.000000  30.0000   
16612  Default   3455.000000               hannity   3455.000000  29.0000   
21026  Default  15726.000000                  know  15726.000000  28.0000   
16174  Default   6891.000000               gutfeld   6891.000000  27.0000   
31794  Default  16976.000000             president  16976.000000  26.0000   
27885  Default   2949.000000                 north   2949.000000  25.0000   
46830  Default   5369.000000              williams   5369.000000  24.0000   
35131  Default  10882.000000                 right  10882.000000  23.0000   
41027  Default  17930.000000                 think  17930.000000  22.0000   
30229  Default   4705.000000                perino   4705.000000  21.0000   
16016  Default   4333.000000             guilfoyle   4333.000000  20.0000   
46158  Default   4169.000000               watters   4169.000000  19.0000   
23021  Default  11692.000000                  like  11692.000000  18.0000   
25650  Default   2989.000000                  meet   2989.000000  17.0000   
21628  Default   1915.000000                 korea   1915.000000  16.0000   
48024  Default   5779.000000                   yes   5779.000000  15.0000   
45580  Default   8730.000000                  want   8730.000000  14.0000   
2025   Default   4805.000000              american   4805.000000  13.0000   
29616  Default  12794.000000                people  12794.000000  12.0000   
4134   Default    975.000000                 black    975.000000  11.0000   
9781   Default   3087.000000                  deal   3087.000000  10.0000   
20844  Default   1507.000000                   kim   1507.000000   9.0000   
27892  Default   1496.000000           north korea   1496.000000   8.0000   
13512  Default   1745.000000                   fbi   1745.000000   7.0000   
9258   Default   2497.000000             crosstalk   2497.000000   6.0000   
8661   Default   4598.000000               country   4598.000000   5.0000   
27959  Default   1384.000000               nuclear   1384.000000   4.0000   
42309  Default   1471.000000               tonight   1471.000000   3.0000   
44572  Default   1637.000000          unidentified   1637.000000   2.0000   
18022  Default   4602.000000                 house   4602.000000   1.0000   
...        ...           ...                   ...           ...      ...   
9108   Topic10      0.566969         createelement      1.219212   7.6624   
44945  Topic10      0.566969                var js      1.219213   7.6624   
11259  Topic10      0.566969       document script      1.219213   7.6624   
14914  Topic10      0.566969  getelementsbytagname      1.219213   7.6624   
14912  Topic10      0.566968        getelementbyid      1.219214   7.6624   
20365  Topic10      0.566969                 js id      1.219216   7.6624   
18376  Topic10      0.566969                 id js      1.219217   7.6624   
20364  Topic10      0.566968      js createelement      1.219218   7.6624   
9109   Topic10      0.566968      createelement js      1.219227   7.6624   
18375  Topic10      1.11941

In [ ]:
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(cnn_trump["processed_text"])

NUM_TOPICS = 10
# Latent Dirichlet Allocation Model
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)